## This notebook computes the contacts between two molecules for the given range and write results separately to each file.  
This notebook uses CPPTRAJ to compute the contacts. The following command is used to compute contacts in this notebook:
- nativecontacts name prot :molA_range  :molB_range distance 5.0 noimage byresidue writecontacts frame_native_contact.dat

In [1]:
import os

#### The below cell contains all the inputs required from user. The inputs required are as follows:
1. Diretory path
2. Topology file
3. Trajectory file
4. Total number of frames in the given trajectory file
5. Molecule A (Protein) residue range
6. Molecule B (RNA) residue range
7. Frame range  you want to consider as native/beginning of trajectory
8. Frame range  you want to consider as non-native/ending of trajectory
9. filename to write the CPPTRAJ script
10. basename or output pattern for output (contact files) 

In [2]:
# All variable inputs are in this cell --> Change values accordingly
working_dir = '/home/hdhondge/MD_Musashi/Prev_Analysis/Annealing/5X3Z_simA/Analysis/'
top_file = 'topology_split.prmtop'  #Topology file --> path relative to working_dir
traj_file = 'reduced_cluster2.6.nc'  # Trajectory file with only frames you want to compute contacts for
total_frames = 15 # total frames present in trajectory

prot_range = '1-78'
nt_range = '79-81'

native_frame_range = range(1, 2) # excluding last number/int
nonnative_frame_range = range(5, 16)  # excluding last number/int

file_name = 'contacts.cpptraj' # filename for cpptraj input
basename_out = '_native_contact.dat'

# Below two files can be used in plotting the contacts
nat_file = 'nat.txt'
nonnat_file = 'nonnat.txt'

#### Below cell contain the function to create a CPPTRAJ script to compute contacts per frame

In [3]:
# funtion to create cpptraj input file
def create_in_cpptraj(topology, trajectory, frame, total_frames, molA_range, molB_range, basename, file):
    with open(file, 'w') as f:
        f.write('parm ' + topology +'\n')
        f.write('trajin ' + trajectory + ' ' + str(frame) + ' last ' + str(total_frames + 100) + '\n')
        f.write('nativecontacts name prot :' + molA_range + ' :' + molB_range + \
                ' distance 5.0 noimage byresidue writecontacts ' + str(frame) + basename + '\n')
        f.write('run' + '\n')
        f.write('exit' + '\n')
    
    return file
    

###### The actual code runs from below cell -->

In [4]:
os.chdir(working_dir)
os.system('conda activate MDsimu')
nat = []
# For native (first couple of frames) 
for a in list(native_frame_range):
    file = create_in_cpptraj(top_file, traj_file, a, total_frames, prot_range, nt_range, basename_out, file_name)
    cmd = 'cpptraj -i '+ file 
    os.system(cmd)
    nat.append(str(a)+basename_out)

# writing all output file names in a single file
with open(nat_file, 'w') as f:
    f.write('\n'.join(nat))

nonnat = []
# For nonnative (last couple of frames) 
for a in list(nonnative_frame_range):
    file = create_in_cpptraj(top_file, traj_file, a, total_frames, prot_range, nt_range, basename_out, file_name)
    cmd = 'cpptraj -i '+ file 
    os.system(cmd)
    nonnat.append(str(a)+basename_out)

# writing all output file names in a single file
with open(nonnat_file, 'w') as f:
    f.write('\n'.join(nonnat))
